### Display the trade details

* Note that using `data.head()` would also work, however most indicators have some "startup" data at the top of the dataframe.
* Some possible problems
    * Columns with NaN values at the end of the dataframe
    * Columns used in `crossed*()` functions with completely different units
* Comparison with full backtest
    * having 200 buy signals as output for one pair from `analyze_ticker()` does not necessarily mean that 200 trades will be made during backtesting.
    * Assuming you use only one condition such as, `df['rsi'] < 30` as buy condition, this will generate multiple "buy" signals for each pair in sequence (until rsi returns > 29). The bot will only buy on the first of these signals (and also only if a trade-slot ("max_open_trades") is still available), or on one of the middle signals, as soon as a "slot" becomes available.  


### Load backtest results to pandas dataframe

Analyze a trades dataframe (also used below for plotting)

In [1]:
from util import ProjectPath

import sys
import csv 
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats
from tabulate import tabulate
import pandas as pd
from prettytable import PrettyTable

class BackTestResults:
    def __init__(self):
        self.all_results = dict()

    def append(self):
        return 

    def sort(self):
        return 
    
    def prettyPrint(self):
        return 

    

class Backtest:

    @staticmethod
    def strategy_comparison_fields():
        return ["key","trades","profit_sum","profit_sum_pct","profit_total_abs","profit_total","profit_total_pct","duration_avg","wins","draws","losses","max_drawdown_account","max_drawdown_abs"]

    def __init__(self,timerange,timeframe,timeframedet,pairs):
        self.__timerange=timeframe
        self.__timeframe=timeframe
        self.__pairs = pairs
        self.__timeframedet=timeframedet
        self.__table= { k:[] for k in Backtest.strategy_comparison_fields()}

    def iterate_by_column(self):
        '''
        key: backtest index (like total profit)
        value: list of result for every strategy
        '''
        for data in self.__table:
            yield data

    def iterate_by_strategy_result_as_dict(self):
        '''
        list of dict
        every dict is backtest format of one result 
        
        '''
        for i in range(len(list(self.__table["key"]))):
            result = {}
            for k,v in self.__table.items():
                result[k] = v[i]
            
            yield result 

    def iterate_by_strategy_result_as_list(self):
        '''
        list of list
        every list is the strategy name followed by results for every comparison field
        '''
        for i in range(len(list(self.__table.keys()))):
            row = []
            for k,v in self.__table.items():
                row.append(v[i])
            yield row 



    def top_result(self):
        return 

    def sort(self,index):
        return

    def pretty_print(self):
        print("============================")
        print(f"Timeframe: {self.__timeframe}, TimeRange: {self.__timerange}, TimeFrame Details: {self.__timeframedet}")
        print("============================")
        print(f"pairs: {self.__pairs}")
        print("============================")
        pt = PrettyTable()
        pt.field_names = list(self.__table.keys())

        for row in self.iterate_by_strategy_result_as_list():
            pt.add_row(row)

        print(pt)

    def save_as_csv(self,path):
        '''
        save backest as a csv  
        '''
        try:
            with open(path, 'w') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=self.__table.keys())
                writer.writeheader()

                for row in self.iterate_by_strategy_result_as_dict():
                    writer.writerow(row)

        except IOError as e:
            print(f"Csv file error {e}")
        return     

    def to_dataframe(self):
        '''
        convert to pandas.DataFrame
        '''
        return 

    def add_strategy_result(self,result_dataframe): #dataframe
        
        for k in Backtest.strategy_comparison_fields():
            self.__table[k].append(result_dataframe[k])


#load strategy
def load_backtest(backtest_results_path):

    def extract(stat,key):
        strategy_name = stat["strategy_comparison"][0]['key']
        return stat['strategy'][strategy_name][key]

    is_init = False
    backtest =Backtest(0,0,0,"")
    nloaded = 0
    nfiles = 0
    for filename in os.listdir(backtest_results_path):
        if filename.endswith(".json") and not filename.endswith(".meta.json"):
            nfiles = nfiles + 1
            loaded_stats = load_backtest_stats(backtest_results_path.joinpath(filename))
            if 'strategy_comparison' in loaded_stats:
                nloaded = nloaded + 1
                if not is_init:
                    timeframe = extract(loaded_stats,'timeframe')
                    timeframe_detail = extract(loaded_stats,'timeframe_detail')
                    timerange = extract(loaded_stats,'timerange')
                    pairs = extract(loaded_stats,'pairlist')

                    backtest = Backtest(timerange,timeframe,timeframe_detail,pairs)
                    is_init = True
            
                backtest.add_strategy_result(loaded_stats["strategy_comparison"][0])

    print(f"scraped files: {nfiles}, loaded {nloaded}")
    return backtest


pp = ProjectPath()

# if backtest_dir points to a directory, it'll automatically load the last backtest file.
test_name = "uptrend_4h"
backtest = load_backtest(pp.backtest_results(test_name))
backtest.save_as_csv(pp.backtest_results(test_name).joinpath("report.csv"))
backtest.pretty_print()


scraped files: 44, loaded 43
Timeframe: 4h, TimeRange: 4h, TimeFrame Details: 5m
pairs: ['AUDIO/USDT', 'AAVE/USDT', 'ALICE/USDT', 'ARPA/USDT', 'AVAX/USDT', 'ATOM/USDT', 'ANKR/USDT', 'AXS/USDT', 'ADA/USDT', 'ALGO/USDT', 'BTS/USDT', 'BAND/USDT', 'BEL/USDT', 'BTC/USDT', 'BLZ/USDT', 'BAT/USDT', 'CHR/USDT', 'C98/USDT', 'COTI/USDT', 'CHZ/USDT', 'COMP/USDT', 'CRV/USDT', 'CELO/USDT', 'DUSK/USDT', 'DOGE/USDT', 'DENT/USDT', 'DASH/USDT', 'DOT/USDT', 'DYDX/USDT', 'ENJ/USDT', 'EOS/USDT', 'ETH/USDT', 'ETC/USDT', 'ENS/USDT', 'EGLD/USDT', 'FIL/USDT', 'FTM/USDT', 'FLM/USDT', 'GRT/USDT', 'GALA/USDT', 'HBAR/USDT', 'HOT/USDT', 'IOTX/USDT', 'ICX/USDT', 'ICP/USDT', 'IOTA/USDT', 'IOST/USDT', 'KLAY/USDT', 'KAVA/USDT', 'KNC/USDT', 'KSM/USDT', 'LRC/USDT', 'LINA/USDT', 'LTC/USDT', 'LINK/USDT', 'MATIC/USDT', 'NEAR/USDT', 'MANA/USDT', 'MTL/USDT', 'NEO/USDT', 'ONT/USDT', 'OMG/USDT', 'OCEAN/USDT', 'OGN/USDT', 'ONE/USDT', 'RLC/USDT', 'RUNE/USDT', 'RVN/USDT', 'RSR/USDT', 'REEF/USDT', 'ROSE/USDT', 'SNX/USDT', 'SAND/USD

In [6]:
# Load backtested trades as dataframe
trades = load_backtest_data(backtest_dir)

# Show value-counts per pair
trades.groupby("pair")["exit_reason"].value_counts()

NameError: name 'backtest_dir' is not defined

## Plotting daily profit / equity line

In [ ]:
# Plotting equity line (starting with 0 on day 1 and adding daily profit for each backtested day)

from freqtrade.configuration import Configuration
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats
import plotly.express as px
import pandas as pd

# strategy = 'SampleStrategy'
# config = Configuration.from_files(["user_data/config.json"])
# backtest_dir = config["user_data_dir"] / "backtest_results"

stats = load_backtest_stats(backtest_dir)
strategy_stats = stats['strategy'][strategy]

dates = []
profits = []
for date_profit in strategy_stats['daily_profit']:
    dates.append(date_profit[0])
    profits.append(date_profit[1])

equity = 0
equity_daily = []
for daily_profit in profits:
    equity_daily.append(equity)
    equity += float(daily_profit)


df = pd.DataFrame({'dates': dates,'equity_daily': equity_daily})

fig = px.line(df, x="dates", y="equity_daily")
fig.show()


### Load live trading results into a pandas dataframe

In case you did already some trading and want to analyze your performance

In [ ]:
from freqtrade.data.btanalysis import load_trades_from_db

# Fetch trades from database
trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
trades.groupby("pair")["exit_reason"].value_counts()

## Analyze the loaded trades for trade parallelism
This can be useful to find the best `max_open_trades` parameter, when used with backtesting in conjunction with `--disable-max-market-positions`.

`analyze_trade_parallelism()` returns a timeseries dataframe with an "open_trades" column, specifying the number of open trades for each candle.

In [ ]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, '5m')

parallel_trades.plot()

## Plot results

Freqtrade offers interactive plotting capabilities based on plotly.

In [ ]:
from freqtrade.plot.plotting import  generate_candlestick_graph
# Limit graph period to keep plotly quick and reactive

# Filter trades to one pair
trades_red = trades.loc[trades['pair'] == pair]

data_red = data['2019-06-01':'2019-06-10']
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades_red,
                                   indicators1=['sma20', 'ema50', 'ema55'],
                                   indicators2=['rsi', 'macd', 'macdsignal', 'macdhist']
                                  )




In [ ]:
# Show graph inline
# graph.show()

# Render graph in a seperate window
graph.show(renderer="browser")


## Plot average profit per trade as distribution graph

In [ ]:
import plotly.figure_factory as ff

hist_data = [trades.profit_ratio]
group_labels = ['profit_ratio']  # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, bin_size=0.01)
fig.show()


Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.